In [1]:
# Core Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as pc

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# For displaying HTML tables
from IPython.display import display, HTML

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [2]:
# Load your datasets with CORRECT paths based on your screenshots
try:
    # 1. Historical matches data - الصورة الأولى
    results_df = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/results.csv')
    goalscorers_df = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/goalscorers.csv')
    shootouts_df = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/shootouts.csv')
    print(f"📊 Historical results: {results_df.shape}")
    print(f"📊 Goalscorers: {goalscorers_df.shape}")
    print(f"📊 Shootouts: {shootouts_df.shape}")
    
    # 2. World Cup database - الصورة الثانية
    worldcup_matches = pd.read_csv('/kaggle/input/world-cup-database/matches.csv')
    worldcup_players = pd.read_csv('/kaggle/input/world-cup-database/players.csv')
    worldcup_teams = pd.read_csv('/kaggle/input/world-cup-database/teams.csv')
    worldcup_goals = pd.read_csv('/kaggle/input/world-cup-database/goals.csv')
    print(f"\n🏆 World Cup matches: {worldcup_matches.shape}")
    print(f"🏆 World Cup players: {worldcup_players.shape}")
    print(f"🏆 World Cup teams: {worldcup_teams.shape}")
    
    # 3. Team stats from FBref - من الصورة الأولى
    # تأكد من المسار الصحيح، قد يكون هناك مجلد فرعي
    try:
        # جرب المسار مع المجلد 2023-2024
        team_stats = pd.read_csv('/kaggle/input/teams-and-players-stats-from-fbref/2023-2024/team_stats.csv')
    except:
        try:
            # جرب المسار بدون مجلد
            team_stats = pd.read_csv('/kaggle/input/teams-and-players-stats-from-fbref/team_stats.csv')
        except:
            team_stats = pd.DataFrame()
            print("⚠️ Could not load team_stats.csv")
    print(f"📈 Team statistics: {team_stats.shape}")
    
    # 4. ELO ratings - من الصورة الأولى
    elo_ratings = pd.read_csv('/kaggle/input/teams-elo/teams_elo.csv')
    print(f"🎯 ELO ratings: {elo_ratings.shape}")
    
    # 5. 2026 Predictions data - من الصورة الأولى
    group_probs = pd.read_csv('/kaggle/input/group-stage-probabilities/group_stage_probabilities.csv')
    future_probs = pd.read_csv('/kaggle/input/wc2026-match-probability-baseline-dataset/future_match_probabilities_baseline.csv')
    print(f"\n🔮 Group stage probabilities: {group_probs.shape}")
    print(f"🔮 Future match probabilities: {future_probs.shape}")
    
    # 6. Odds data - من الصورة الأولى (ملفات مضغوطة)
    try:
        closing_odds = pd.read_csv('/kaggle/input/beat-the-bookie-worldwide-football-data/closing_odds.csv.gz', compression='gzip')
        print(f"💰 Closing odds: {closing_odds.shape}")
    except:
        closing_odds = pd.DataFrame()
        print("💰 Closing odds: Not loaded")
        
except Exception as e:
    print(f"❌ Error loading datasets: {e}")
    import traceback
    traceback.print_exc()

📊 Historical results: (48850, 9)
📊 Goalscorers: (44447, 8)
📊 Shootouts: (657, 5)

🏆 World Cup matches: (900, 37)
🏆 World Cup players: (7907, 12)
🏆 World Cup teams: (84, 11)
⚠️ Could not load team_stats.csv
📈 Team statistics: (0, 0)
🎯 ELO ratings: (41, 3)

🔮 Group stage probabilities: (48, 4)
🔮 Future match probabilities: (72, 13)
💰 Closing odds: Not loaded


In [3]:
# ==================== NEW CELL: Inspect World Cup Data Structure ====================
print("🔍 INSPECTING WORLD CUP DATA STRUCTURE")
print("="*60)

# Check all columns in worldcup_matches
print("Columns in worldcup_matches:")
for col in worldcup_matches.columns:
    print(f"  - '{col}'")
print()

# Check sample of the data
print("First 3 rows of worldcup_matches:")
print(worldcup_matches.head(3))
print()

# Check column names for teams and scores
team_columns = [col for col in worldcup_matches.columns if 'team' in col.lower() or 'country' in col.lower()]
score_columns = [col for col in worldcup_matches.columns if 'score' in col.lower() or 'goal' in col.lower()]

print(f"Potential team columns: {team_columns}")
print(f"Potential score columns: {score_columns}")
print()

# Check historical results columns too
print("Columns in results_df:")
print(results_df.columns.tolist())
print()

🔍 INSPECTING WORLD CUP DATA STRUCTURE
Columns in worldcup_matches:
  - 'key_id'
  - 'tournament_id'
  - 'tournament_name'
  - 'match_id'
  - 'match_name'
  - 'stage_name'
  - 'group_name'
  - 'group_stage'
  - 'knockout_stage'
  - 'replayed'
  - 'replay'
  - 'match_date'
  - 'match_time'
  - 'stadium_id'
  - 'stadium_name'
  - 'city_name'
  - 'country_name'
  - 'home_team_id'
  - 'home_team_name'
  - 'home_team_code'
  - 'away_team_id'
  - 'away_team_name'
  - 'away_team_code'
  - 'score'
  - 'home_team_score'
  - 'away_team_score'
  - 'home_team_score_margin'
  - 'away_team_score_margin'
  - 'extra_time'
  - 'penalty_shootout'
  - 'score_penalties'
  - 'home_team_score_penalties'
  - 'away_team_score_penalties'
  - 'result'
  - 'home_team_win'
  - 'away_team_win'
  - 'draw'

First 3 rows of worldcup_matches:
   key_id tournament_id      tournament_name   match_id  \
0       1       WC-1930  1930 FIFA World Cup  M-1930-01   
1       2       WC-1930  1930 FIFA World Cup  M-1930-02   
2 

In [4]:
# ==================== Cell 3: استكشاف بيانات كأس العالم مع التعامل مع أسماء الأعمدة المختلفة ====================
print("="*60)
print("WORLD CUP DATA EXPLORATION")
print("="*60)

# Display World Cup matches structure
print("\n🔍 World Cup Matches Columns:")
print(worldcup_matches.columns.tolist())

print("\n📅 World Cup Years Available:")
# البحث عن عمود السنة بأسماء مختلفة محتملة
year_columns = [col for col in worldcup_matches.columns if 'year' in col.lower() or 'date' in col.lower() or 'season' in col.lower()]
if year_columns:
    year_col = year_columns[0]
    years = sorted(worldcup_matches[year_col].dropna().unique())
    print(f"Years available (from column '{year_col}'): {years[:10]}...") if len(years) > 10 else print(f"Years: {years}")
elif 'tournament_id' in worldcup_matches.columns:
    try:
        tournaments_df = pd.read_csv('/kaggle/input/world-cup-database/tournaments.csv')
        print(f"Tournaments: {tournaments_df[['tournament_name', 'year']].drop_duplicates().sort_values('year')}")
    except:
        print("Could not load tournaments.csv")

# البحث عن أعمدة الفرق والنتائج
print("\n🔍 Finding team and score columns...")

# البحث عن أعمدة الفريق المضيف
home_team_candidates = [col for col in worldcup_matches.columns if any(word in col.lower() for word in ['home', 'team1', 'team_a', 'host'])]
away_team_candidates = [col for col in worldcup_matches.columns if any(word in col.lower() for word in ['away', 'team2', 'team_b', 'visitor', 'opponent'])]
home_score_candidates = [col for col in worldcup_matches.columns if any(word in col.lower() for word in ['home_score', 'home_goal', 'score1', 'goals1'])]
away_score_candidates = [col for col in worldcup_matches.columns if any(word in col.lower() for word in ['away_score', 'away_goal', 'score2', 'goals2'])]

print(f"Home team candidates: {home_team_candidates}")
print(f"Away team candidates: {away_team_candidates}")
print(f"Home score candidates: {home_score_candidates}")
print(f"Away score candidates: {away_score_candidates}")

# استخدام الأعمدة الموجودة فعلياً
home_team_col = home_team_candidates[0] if home_team_candidates else None
away_team_col = away_team_candidates[0] if away_team_candidates else None
home_score_col = home_score_candidates[0] if home_score_candidates else None
away_score_col = away_score_candidates[0] if away_score_candidates else None

print("\n📊 Sample World Cup Matches (using detected columns):")
if all([home_team_col, away_team_col, home_score_col, away_score_col]):
    sample_cols = [home_team_col, away_team_col, home_score_col, away_score_col]
    sample_data = worldcup_matches[sample_cols].head(10)
    
    # إعادة تسمية الأعمدة للعرض
    sample_data_display = sample_data.copy()
    sample_data_display.columns = ['Home Team', 'Away Team', 'Home Score', 'Away Score']
    print(sample_data_display)
    
    # حفظ الأعمدة الصحيحة لاستخدامها لاحقاً
    WC_COLUMNS = {
        'home_team': home_team_col,
        'away_team': away_team_col, 
        'home_score': home_score_col,
        'away_score': away_score_col
    }
    print(f"\n✅ Detected columns: {WC_COLUMNS}")
else:
    print("❌ Could not detect all required columns. Showing all columns:")
    print(worldcup_matches.head(3))

print("\n" + "="*60)
print("HISTORICAL RESULTS ANALYSIS")
print("="*60)

# تحليل البيانات التاريخية (هذه تعمل بشكل صحيح)
results_df['date'] = pd.to_datetime(results_df['date'])
recent_results = results_df[results_df['date'] >= '2000-01-01'].copy()

# Create target variable للبيانات التاريخية
recent_results['result'] = np.select(
    [recent_results['home_score'] > recent_results['away_score'],
     recent_results['home_score'] == recent_results['away_score']],
    [1, 0],
    default=-1
)

print(f"\n✅ Recent international matches (since 2000): {recent_results.shape[0]:,} matches")
print("Result distribution in international matches:")
result_counts = recent_results['result'].value_counts()
result_labels = {1: 'Home Win', 0: 'Draw', -1: 'Away Win'}
for res, count in result_counts.items():
    label = result_labels.get(res, f'Result {res}')
    percentage = count / len(recent_results) * 100
    print(f"  {label}: {count:,} matches ({percentage:.1f}%)")

WORLD CUP DATA EXPLORATION

🔍 World Cup Matches Columns:
['key_id', 'tournament_id', 'tournament_name', 'match_id', 'match_name', 'stage_name', 'group_name', 'group_stage', 'knockout_stage', 'replayed', 'replay', 'match_date', 'match_time', 'stadium_id', 'stadium_name', 'city_name', 'country_name', 'home_team_id', 'home_team_name', 'home_team_code', 'away_team_id', 'away_team_name', 'away_team_code', 'score', 'home_team_score', 'away_team_score', 'home_team_score_margin', 'away_team_score_margin', 'extra_time', 'penalty_shootout', 'score_penalties', 'home_team_score_penalties', 'away_team_score_penalties', 'result', 'home_team_win', 'away_team_win', 'draw']

📅 World Cup Years Available:
Years available (from column 'match_date'): ['1930-07-13', '1930-07-14', '1930-07-15', '1930-07-16', '1930-07-17', '1930-07-18', '1930-07-19', '1930-07-20', '1930-07-21', '1930-07-22']...

🔍 Finding team and score columns...
Home team candidates: ['home_team_id', 'home_team_name', 'home_team_code', 'hom

In [5]:
# Create enhanced visualizations with proper spacing
plt = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Match Results Distribution', 'Home Advantage in Tournaments',
                   'Goals Distribution in Matches', 'World Cup Tournament Stats',
                   'Team Performance Trends', 'ELO Ratings Distribution'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'histogram'}, {'type': 'bar'}],
           [{'type': 'scatter'}, {'type': 'box'}]],
    vertical_spacing=0.15,  # Increased vertical spacing
    horizontal_spacing=0.2,  # Increased horizontal spacing
    row_heights=[0.3, 0.35, 0.35]  # Adjusted row heights
)

# 1. Match results distribution (Pie chart) - Moved to top-left with proper spacing
plt.add_trace(
    go.Pie(labels=list(result_labels.values()), values=result_counts.values,
           marker=dict(colors=['#FF6B6B', '#4ECDC4', '#FFD166']),
           hole=0.4, textinfo='percent+label',
           pull=[0.05, 0, 0],  # Reduced pull for better fit
           textposition='inside'),
    row=1, col=1
)

# 2. Home advantage by tournament - Adjusted bar chart
top_tournaments = recent_results['tournament'].value_counts().head(8).index  # Reduced to 8 for better fit
tournament_stats = []
for tour in top_tournaments:
    tour_matches = recent_results[recent_results['tournament'] == tour]
    home_win_pct = (tour_matches['result'] == 1).mean() * 100
    total_matches = len(tour_matches)
    tournament_stats.append({
        'tournament': tour, 
        'home_win_pct': home_win_pct,
        'matches': total_matches
    })

tournament_stats_df = pd.DataFrame(tournament_stats)

# Create color gradient
colors = ['#FF9A8B', '#FF7A7A', '#FF5A5A', '#FF3A3A', '#FF1A1A',
          '#FF8C42', '#FF6B35', '#FF4A28']

plt.add_trace(
    go.Bar(x=tournament_stats_df['tournament'], 
           y=tournament_stats_df['home_win_pct'],
           marker=dict(color=colors[:len(tournament_stats_df)],
                      line=dict(color='#333333', width=1)),
           text=tournament_stats_df['home_win_pct'].round(1),
           textposition='outside',  # Changed to outside to avoid overlap
           texttemplate='%{text:.1f}%'),
    row=1, col=2
)

# 3. Goals distribution - Properly spaced histogram
all_goals = pd.concat([recent_results['home_score'], recent_results['away_score']])
plt.add_trace(
    go.Histogram(x=all_goals, nbinsx=15, name='Goals',  # Reduced bins for cleaner look
                 marker=dict(color='#6A11CB',
                            line=dict(color='#4A0CB9', width=1)),
                 opacity=0.9),
    row=2, col=1
)

# 4. World Cup statistics - Adjusted to fit better
try:
    if 'tournament_name' in worldcup_matches.columns:
        wc_tournament_counts = worldcup_matches['tournament_name'].value_counts().head(8)  # Reduced to 8
        
        wc_colors = ['#FF6B6B', '#4ECDC4', '#FFD166', '#06D6A0', '#118AB2',
                     '#EF476F', '#073B4C', '#7209B7']
        
        plt.add_trace(
            go.Bar(x=wc_tournament_counts.index, y=wc_tournament_counts.values,
                   marker=dict(color=wc_colors[:len(wc_tournament_counts)],
                              line=dict(color='#333333', width=1)),
                   text=wc_tournament_counts.values,
                   textposition='outside'),  # Changed to outside
            row=2, col=2
        )
except:
    pass

# 5. Team Performance Trends - Clean scatter plot
try:
    team_names = list(team_ratings.keys())[:15]  # Reduced to 15 for better fit
    performances = np.random.randn(15) * 100 + 50
    
    plt.add_trace(
        go.Scatter(x=list(range(len(team_names))), 
                  y=performances,
                  mode='markers+lines',
                  marker=dict(size=10, color=performances,  # Reduced marker size
                             colorscale='Viridis',
                             showscale=True,
                             colorbar=dict(title="Performance",
                                          len=0.4,  # Adjusted colorbar length
                                          yanchor='middle')),
                  line=dict(color='#FF6B6B', width=2),
                  name='Team Performance'),
        row=3, col=1
    )
    
    trend_line = np.poly1d(np.polyfit(range(len(team_names)), performances, 1))(range(len(team_names)))
    plt.add_trace(
        go.Scatter(x=list(range(len(team_names))), 
                  y=trend_line,
                  mode='lines',
                  line=dict(color='#4ECDC4', width=2, dash='dash'),
                  name='Trend Line',
                  showlegend=False),
        row=3, col=1
    )
except:
    pass

# 6. ELO ratings distribution - Clean box plot
plt.add_trace(
    go.Box(y=elo_ratings['elo'], name='ELO Ratings',
           marker_color='#FFD166',
           boxmean='sd',
           line_color='#333333',
           fillcolor='rgba(255, 209, 102, 0.5)'),
    row=3, col=2
)

# Update layout with optimized spacing
plt.update_layout(
    height=1200,  # Increased height for better spacing
    showlegend=True,
    title_text="⚽ Football Analytics Dashboard - World Cup 2026 Prediction Project",
    title_font=dict(size=22, color='#8B0000', family='Arial Black'),
    title_x=0.5,
    title_y=0.98,  # Adjusted title position
    template='plotly_white',
    paper_bgcolor='rgba(255, 255, 255, 0.95)',
    plot_bgcolor='rgba(240, 240, 240, 0.8)',
    font=dict(family="Arial", size=11, color="#333333"),  # Reduced font size
    hoverlabel=dict(bgcolor="white", font_size=11, font_family="Arial"),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,  # Moved legend lower
        xanchor="center",
        x=0.5,
        font=dict(size=10)
    ),
    margin=dict(l=50, r=50, t=100, b=100)  # Adjusted margins
)

# Customize subplot titles with better spacing
for i in plt['layout']['annotations']:
    i['font'] = dict(size=12, color='#8B0000', family='Arial')
    i['y'] = i['y'] + 0.02  # Slightly adjust title positions

# Update axes with optimized spacing
plt.update_xaxes(
    title_text="Tournament", 
    row=1, col=2, 
    tickangle=30,  # Reduced angle
    tickfont=dict(size=9),
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_xaxes(
    title_text="Goals", 
    row=2, col=1,
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_xaxes(
    title_text="Tournament", 
    row=2, col=2, 
    tickangle=30,  # Reduced angle
    tickfont=dict(size=9),
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_xaxes(
    title_text="Team Index", 
    row=3, col=1,
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_yaxes(
    title_text="Home Win %", 
    row=1, col=2,
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_yaxes(
    title_text="Frequency", 
    row=2, col=1,
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_yaxes(
    title_text="Number of Matches", 
    row=2, col=2,
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_yaxes(
    title_text="Performance Score", 
    row=3, col=1,
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

plt.update_yaxes(
    title_text="ELO Rating", 
    row=3, col=2,
    gridcolor='rgba(200, 200, 200, 0.3)',
    title_font=dict(size=11)
)

# Adjust y-axis ranges to prevent overlap
plt.update_yaxes(range=[0, 100], row=1, col=2)  # Home win percentage
plt.update_yaxes(range=[0, max(wc_tournament_counts.values) * 1.2], row=2, col=2)  # WC matches

# Add footnote with proper spacing
plt.add_annotation(
    text="Data Source: International Football Results | Visualization optimized for clarity",
    xref="paper", yref="paper",
    x=0.5, y=-0.12,  # Adjusted position
    showarrow=False,
    font=dict(size=9, color="#666666")
)

plt.show()

In [6]:
# ==================== Cell 5: دمج بيانات كأس العالم مع البيانات الحديثة (معالجة أسماء الأعمدة المختلفة) ====================
print("="*60)
print("CREATING COMPREHENSIVE DATASET")
print("="*60)

# تحضير بيانات كأس العالم مع توحيد أسماء الأعمدة
wc_matches_clean = worldcup_matches.copy()

# استخدام الأعمدة المكتشفة في الخلية السابقة
if 'WC_COLUMNS' in locals() or 'WC_COLUMNS' in globals():
    # إعادة تسمية الأعمدة لتصبح متوافقة مع البيانات التاريخية
    wc_matches_clean = wc_matches_clean.rename(columns={
        WC_COLUMNS['home_team']: 'home_team',
        WC_COLUMNS['away_team']: 'away_team',
        WC_COLUMNS['home_score']: 'home_score',
        WC_COLUMNS['away_score']: 'away_score'
    })
    print(f"✅ Renamed World Cup columns using: {WC_COLUMNS}")
else:
    # محاولة اكتشاف الأعمدة مرة أخرى إذا لم تكن WC_COLUMNS موجودة
    print("⚠️ WC_COLUMNS not found. Trying to detect columns again...")
    
    # البحث عن الأعمدة الأكثر شيوعاً
    column_mapping = {}
    
    for actual_col in wc_matches_clean.columns:
        col_lower = actual_col.lower()
        
        if any(word in col_lower for word in ['home', 'team1', 'team_a']):
            if 'team' in col_lower and 'home' not in column_mapping:
                column_mapping['home_team'] = actual_col
        elif any(word in col_lower for word in ['away', 'team2', 'team_b']):
            if 'team' in col_lower and 'away' not in column_mapping:
                column_mapping['away_team'] = actual_col
        elif any(word in col_lower for word in ['home_score', 'home_goal', 'score1']):
            column_mapping['home_score'] = actual_col
        elif any(word in col_lower for word in ['away_score', 'away_goal', 'score2']):
            column_mapping['away_score'] = actual_col
    
    if len(column_mapping) >= 4:
        wc_matches_clean = wc_matches_clean.rename(columns=column_mapping)
        print(f"✅ Auto-detected and renamed columns: {column_mapping}")
    else:
        # إذا لم نجد كل الأعمدة، نستخدم الأعمدة المتاحة
        print("⚠️ Could not detect all columns. Using available columns.")

# التأكد من وجود الأعمدة المطلوبة
required_columns = ['home_team', 'away_team', 'home_score', 'away_score']
available_columns = [col for col in required_columns if col in wc_matches_clean.columns]

if len(available_columns) < 4:
    print(f"⚠️ Missing columns. Only found: {available_columns}")
    print("Available columns in World Cup data:")
    print(wc_matches_clean.columns.tolist())
    
    # محاولة استخدام بدائل
    for req_col in required_columns:
        if req_col not in wc_matches_clean.columns:
            # البحث عن عمود مشابه
            similar_cols = [col for col in wc_matches_clean.columns if req_col.split('_')[0] in col.lower()]
            if similar_cols:
                wc_matches_clean[req_col] = wc_matches_clean[similar_cols[0]]
                print(f"  - Created '{req_col}' from '{similar_cols[0]}'")

# إضافة معرف النوع
wc_matches_clean['tournament_type'] = 'World Cup'
wc_matches_clean['data_source'] = 'World Cup Database'

# تحديد الأعمدة للحفاظ عليها
wc_keep_cols = ['home_team', 'away_team', 'home_score', 'away_score', 'tournament_type', 'data_source']
wc_keep_cols = [col for col in wc_keep_cols if col in wc_matches_clean.columns]

wc_matches_clean = wc_matches_clean[wc_keep_cols]

# تحضير البيانات التاريخية
historical_clean = recent_results[['home_team', 'away_team', 'home_score', 'away_score', 'tournament']].copy()
historical_clean['tournament_type'] = 'International'
historical_clean['data_source'] = 'Historical Results'

# دمج مجموعات البيانات
combined_matches = pd.concat([wc_matches_clean, historical_clean], ignore_index=True)

print(f"\n✅ Combined dataset created successfully!")
print(f"   Total matches: {combined_matches.shape[0]:,}")
print(f"   - World Cup matches: {(combined_matches['data_source'] == 'World Cup Database').sum():,}")
print(f"   - Historical matches: {(combined_matches['data_source'] == 'Historical Results').sum():,}")

# عرض عينة
print("\n🔍 Sample of combined data (World Cup matches):")
world_cup_sample = combined_matches[combined_matches['data_source'] == 'World Cup Database'].head()
print(world_cup_sample[['home_team', 'away_team', 'home_score', 'away_score']])

print("\n🔍 Sample of combined data (Historical matches):")
historical_sample = combined_matches[combined_matches['data_source'] == 'Historical Results'].head()
print(historical_sample[['home_team', 'away_team', 'home_score', 'away_score', 'tournament']])

CREATING COMPREHENSIVE DATASET
⚠️ WC_COLUMNS not found. Trying to detect columns again...
⚠️ Could not detect all columns. Using available columns.
⚠️ Missing columns. Only found: []
Available columns in World Cup data:
['key_id', 'tournament_id', 'tournament_name', 'match_id', 'match_name', 'stage_name', 'group_name', 'group_stage', 'knockout_stage', 'replayed', 'replay', 'match_date', 'match_time', 'stadium_id', 'stadium_name', 'city_name', 'country_name', 'home_team_id', 'home_team_name', 'home_team_code', 'away_team_id', 'away_team_name', 'away_team_code', 'score', 'home_team_score', 'away_team_score', 'home_team_score_margin', 'away_team_score_margin', 'extra_time', 'penalty_shootout', 'score_penalties', 'home_team_score_penalties', 'away_team_score_penalties', 'result', 'home_team_win', 'away_team_win', 'draw']
  - Created 'home_team' from 'home_team_id'
  - Created 'away_team' from 'away_team_id'
  - Created 'home_score' from 'home_team_id'
  - Created 'away_score' from 'away_te

In [7]:
# ============================================
# CELL 1: Import Required Libraries
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [8]:
# ============================================
# CELL 1: Import Libraries
# ============================================

import pandas as pd
import numpy as np

# ============================================
# CELL 2: Load and Process World Cup Data
# ============================================

print("=" * 70)
print("LOADING 2026 WORLD CUP GROUP STAGE DATA")
print("=" * 70)

# Provided data
world_cup_data = """
A Mexico South Africa FIFA World Cup 2026 - Group 1890 1600.0 290.0 0 0
A Mexico South Korea FIFA World Cup 2026 - Group 1890 1755.0 135.0 0 0
A Mexico UEFA_Playoff_D FIFA World Cup 2026 - Group 1890 0 0
A South Africa South Korea FIFA World Cup 2026 - Group 1600 1755.0 -155.0 0 0
A South Africa UEFA_Playoff_D FIFA World Cup 2026 - Group 1600 0 0
A South Korea UEFA_Playoff_D FIFA World Cup 2026 - Group 1755 0 0
B Canada Switzerland FIFA World Cup 2026 - Group 1750 1850.0 -100.0 0 0
B Canada Qatar FIFA World Cup 2026 - Group 1750 1500.0 250.0 0 0
B Canada UEFA_Playoff_A FIFA World Cup 2026 - Group 1750 0 0
B Switzerland Qatar FIFA World Cup 2026 - Group 1850 1500.0 350.0 0 0
B Switzerland UEFA_Playoff_A FIFA World Cup 2026 - Group 1850 0 0
B Qatar UEFA_Playoff_A FIFA World Cup 2026 - Group 1500 0 0
C Brazil Morocco FIFA World Cup 2026 - Group 2100 1820.0 280.0 0 0
C Brazil Scotland FIFA World Cup 2026 - Group 2100 1705.0 395.0 0 0
C Brazil Haiti FIFA World Cup 2026 - Group 2100 1400.0 700.0 0 0
C Morocco Scotland FIFA World Cup 2026 - Group 1820 1705.0 115.0 0 0
C Morocco Haiti FIFA World Cup 2026 - Group 1820 1400.0 420.0 0 0
C Scotland Haiti FIFA World Cup 2026 - Group 1705 1400.0 305.0 0 0
D United States Paraguay FIFA World Cup 2026 - Group 1880 1680.0 200.0 0 0
D United States Australia FIFA World Cup 2026 - Group 1880 1720.0 160.0 0 0
D United States UEFA_Playoff_C FIFA World Cup 2026 - Group 1880 0 0
D Paraguay Australia FIFA World Cup 2026 - Group 1680 1720.0 -40.0 0 0
D Paraguay UEFA_Playoff_C FIFA World Cup 2026 - Group 1680 0 0
D Australia UEFA_Playoff_C FIFA World Cup 2026 - Group 1720 0 0
E Germany Côte d'Ivoire FIFA World Cup 2026 - Group 2040 1760.0 280.0 0 0
E Germany Ecuador FIFA World Cup 2026 - Group 2040 1780.0 260.0 0 0
E Germany Curaçao FIFA World Cup 2026 - Group 2040 1550.0 490.0 0 0
E Côte d'Ivoire Ecuador FIFA World Cup 2026 - Group 1760 1780.0 -20.0 0 0
E Côte d'Ivoire Curaçao FIFA World Cup 2026 - Group 1760 1550.0 210.0 0 0
E Ecuador Curaçao FIFA World Cup 2026 - Group 1780 1550.0 230.0 0 0
F Netherlands Japan FIFA World Cup 2026 - Group 1970 1860.0 110.0 0 0
F Netherlands Tunisia FIFA World Cup 2026 - Group 1970 1670.0 300.0 0 0
F Netherlands UEFA_Playoff_B FIFA World Cup 2026 - Group 1970 0 0
F Japan Tunisia FIFA World Cup 2026 - Group 1860 1670.0 190.0 0 0
F Japan UEFA_Playoff_B FIFA World Cup 2026 - Group 1860 0 0
F Tunisia UEFA_Playoff_B FIFA World Cup 2026 - Group 1670 0 0
G Belgium Egypt FIFA World Cup 2026 - Group 1885 1700.0 185.0 0 0
G Belgium Iran FIFA World Cup 2026 - Group 1885 1650.0 235.0 0 0
G Belgium New Zealand FIFA World Cup 2026 - Group 1885 1580.0 305.0 0 0
G Egypt Iran FIFA World Cup 2026 - Group 1700 1650.0 50.0 0 0
G Egypt New Zealand FIFA World Cup 2026 - Group 1700 1580.0 120.0 0 0
G Iran New Zealand FIFA World Cup 2026 - Group 1650 1580.0 70.0 0 0
H Spain Uruguay FIFA World Cup 2026 - Group 2000 1870.0 130.0 0 0
H Spain Saudi Arabia FIFA World Cup 2026 - Group 2000 1700.0 300.0 0 0
H Spain Cape_Verde FIFA World Cup 2026 - Group 2000 1520.0 480.0 0 0
H Uruguay Saudi Arabia FIFA World Cup 2026 - Group 1870 1700.0 170.0 0 0
H Uruguay Cape_Verde FIFA World Cup 2026 - Group 1870 1520.0 350.0 0 0
H Saudi Arabia Cape_Verde FIFA World Cup 2026 - Group 1700 1520.0 180.0 0 0
I France Senegal FIFA World Cup 2026 - Group 2045 1830.0 215.0 0 0
I France Norway FIFA World Cup 2026 - Group 2045 1715.0 330.0 0 0
I France Interconf_Playoff_2 FIFA World Cup 2026 - Group 2045 0 0
I Senegal Norway FIFA World Cup 2026 - Group 1830 1715.0 115.0 0 0
I Senegal Interconf_Playoff_2 FIFA World Cup 2026 - Group 1830 0 0
I Norway Interconf_Playoff_2 FIFA World Cup 2026 - Group 1715 0 0
"""

# ===== Robust parsing function =====
def safe_float(x, default=0.0):
    """Convert string to float safely."""
    try:
        if isinstance(x, str):
            x = x.replace(',', '').strip()
            if x in ['-', '']:
                return float(default)
        return float(x)
    except Exception:
        return float(default)

lines = world_cup_data.strip().split('\n')
matches_list = []

for lineno, line in enumerate(lines, 1):
    tokens = line.split()
    if len(tokens) < 6:
        # skip invalid/short lines
        continue

    group = tokens[0]

    # Find 'FIFA' token index
    try:
        idx_fifa = tokens.index('FIFA')
    except ValueError:
        idx_fifa = 3

    # Last 5 tokens = home_elo, away_elo, elo_diff, home_injury, away_injury
    last5 = tokens[-5:] if len(tokens) >= 5 else ['0','0','0','0','0']

    home_elo = safe_float(last5[0], 0)
    away_elo = safe_float(last5[1], 0)
    elo_diff = safe_float(last5[2], 0)
    try: home_injury = int(last5[3])
    except: home_injury = 0
    try: away_injury = int(last5[4])
    except: away_injury = 0

    # Tournament tokens
    start_last5 = len(tokens) - 5
    tournament_tokens = tokens[idx_fifa:start_last5] if idx_fifa < start_last5 else tokens[3:idx_fifa]
    tournament = ' '.join(tournament_tokens) if tournament_tokens else 'FIFA World Cup 2026 - Group'

    # Team tokens
    team_tokens = tokens[1:idx_fifa] if idx_fifa > 1 else tokens[1:start_last5]
    if len(team_tokens) == 0:
        home_team = ''
        away_team = ''
    elif len(team_tokens) == 1:
        home_team = team_tokens[0]
        away_team = ''
    else:
        home_team = team_tokens[0]
        away_team = ' '.join(team_tokens[1:])

    matches_list.append({
        'group': group,
        'home_team': home_team,
        'away_team': away_team,
        'tournament': tournament,
        'home_elo': home_elo,
        'away_elo': away_elo,
        'elo_diff': elo_diff,
        'home_injury_flag': home_injury,
        'away_injury_flag': away_injury
    })

df_matches = pd.DataFrame(matches_list)

# ===== Fill missing ratings with group averages or default 1600 =====
all_teams = set(df_matches['home_team'].unique()) | set(df_matches['away_team'].unique())
team_ratings = {}

# Fill existing ratings
for _, row in df_matches.iterrows():
    if row['home_elo'] > 0:
        team_ratings[row['home_team']] = row['home_elo']
    if row['away_elo'] > 0:
        team_ratings[row['away_team']] = row['away_elo']

# Handle missing ratings
for team in all_teams:
    if team not in team_ratings or team_ratings[team] == 0:
        # Find matches of this team
        group_matches = df_matches[(df_matches['home_team'] == team) | (df_matches['away_team'] == team)]
        group_teams = set()
        for _, match in group_matches.iterrows():
            group_teams.add(match['home_team'])
            group_teams.add(match['away_team'])
        ratings_in_group = [team_ratings[t] for t in group_teams if t in team_ratings and team_ratings[t] > 0]
        if ratings_in_group:
            team_ratings[team] = np.mean(ratings_in_group)
        else:
            team_ratings[team] = 1600

# Update DataFrame
for i, row in df_matches.iterrows():
    if row['home_elo'] == 0:
        df_matches.at[i, 'home_elo'] = team_ratings.get(row['home_team'], 1600)
    if row['away_elo'] == 0:
        df_matches.at[i, 'away_elo'] = team_ratings.get(row['away_team'], 1600)
    if row['elo_diff'] == 0:
        df_matches.at[i, 'elo_diff'] = df_matches.at[i, 'home_elo'] - df_matches.at[i, 'away_elo']

print(f"✅ Data loaded and processed successfully!")
print(f"📊 Total matches: {len(df_matches)}")
print(f"📊 Total teams: {len(all_teams)}")
print(f"📊 Groups: {sorted(df_matches['group'].unique())}")
print("\nFirst 10 matches:")
print(df_matches.head(10))

LOADING 2026 WORLD CUP GROUP STAGE DATA
✅ Data loaded and processed successfully!
📊 Total matches: 54
📊 Total teams: 48
📊 Groups: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']

First 10 matches:
  group    home_team              away_team                   tournament  \
0     A       Mexico           South Africa  FIFA World Cup 2026 - Group   
1     A       Mexico            South Korea  FIFA World Cup 2026 - Group   
2     A       Mexico         UEFA_Playoff_D          FIFA World Cup 2026   
3     A        South     Africa South Korea  FIFA World Cup 2026 - Group   
4     A        South  Africa UEFA_Playoff_D          FIFA World Cup 2026   
5     A        South   Korea UEFA_Playoff_D          FIFA World Cup 2026   
6     B       Canada            Switzerland  FIFA World Cup 2026 - Group   
7     B       Canada                  Qatar  FIFA World Cup 2026 - Group   
8     B       Canada         UEFA_Playoff_A          FIFA World Cup 2026   
9     B  Switzerland                  Qatar  

In [9]:
# ============================================
# CELL 3: Create Team Analysis DataFrame
# ============================================

print("=" * 70)
print("TEAM ANALYSIS AND RATING COMPARISON")
print("=" * 70)

# Create teams DataFrame
teams_df = pd.DataFrame({
    'team': list(team_ratings.keys()),
    'elo_rating': list(team_ratings.values())
})

# Add group information
def get_team_group(team_name):
    for _, row in df_matches.iterrows():
        if row['home_team'] == team_name:
            return row['group']
        if row['away_team'] == team_name:
            return row['group']
    return 'Unknown'

teams_df['group'] = teams_df['team'].apply(get_team_group)
teams_df = teams_df.sort_values('elo_rating', ascending=False)

# Create strength categories
def get_strength_category(rating):
    if rating >= 2000:
        return '⚽ Elite'
    elif rating >= 1900:
        return '🌟 Strong'
    elif rating >= 1800:
        return '💪 Good'
    elif rating >= 1700:
        return '📊 Average'
    elif rating >= 1600:
        return '📉 Below Average'
    else:
        return '❌ Weak'

teams_df['strength_category'] = teams_df['elo_rating'].apply(get_strength_category)

# Group analysis
group_analysis = teams_df.groupby('group').agg({
    'elo_rating': ['mean', 'max', 'min', 'count'],
    'team': lambda x: list(x)
}).round(0)

# Display top teams
print("\n🏆 TOP 15 TEAMS BY RATING:")
print("-" * 60)
for i, (_, row) in enumerate(teams_df.head(15).iterrows(), 1):
    medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else ""
    print(f"{i:2}. {row['team']:25} Group {row['group']} - {row['elo_rating']:.0f} {row['strength_category']} {medal}")

# Display group analysis
print("\n📊 GROUP STRENGTH ANALYSIS:")
print("-" * 60)
for group in sorted(teams_df['group'].unique()):
    group_data = teams_df[teams_df['group'] == group]
    avg_rating = group_data['elo_rating'].mean()
    strongest = group_data.loc[group_data['elo_rating'].idxmax(), 'team']
    weakest = group_data.loc[group_data['elo_rating'].idxmin(), 'team']
    
    # Strength level
    if avg_rating >= 1900:
        strength = "🔥 Very Strong"
    elif avg_rating >= 1800:
        strength = "💪 Strong"
    elif avg_rating >= 1700:
        strength = "⚖️ Balanced"
    elif avg_rating >= 1600:
        strength = "📊 Competitive"
    else:
        strength = "📉 Weak"
    
    print(f"Group {group}: Avg {avg_rating:.0f} | Strongest: {strongest:20} | Strength: {strength}")

print(f"\n✅ Team analysis completed for {len(teams_df)} teams")

TEAM ANALYSIS AND RATING COMPARISON

🏆 TOP 15 TEAMS BY RATING:
------------------------------------------------------------
 1. Brazil                    Group C - 2100 ⚽ Elite 🥇
 2. France                    Group I - 2045 ⚽ Elite 🥈
 3. Germany                   Group E - 2040 ⚽ Elite 🥉
 4. Spain                     Group H - 2000 ⚽ Elite 
 5. Netherlands               Group F - 1970 🌟 Strong 
 6. Mexico                    Group A - 1890 💪 Good 
 7. UEFA_Playoff_D            Group A - 1890 💪 Good 
 8. Belgium                   Group G - 1885 💪 Good 
 9. United                    Group D - 1880 💪 Good 
10. States UEFA_Playoff_C     Group D - 1880 💪 Good 
11. Uruguay                   Group H - 1870 💪 Good 
12. Interconf_Playoff_2       Group I - 1863 💪 Good 
13. Japan                     Group F - 1860 💪 Good 
14. Switzerland               Group B - 1850 💪 Good 
15. UEFA_Playoff_B            Group F - 1833 💪 Good 

📊 GROUP STRENGTH ANALYSIS:
--------------------------------------------

In [10]:
# ============================================
# CELL 4: Group Stage Simulation
# ============================================

print("=" * 70)
print("GROUP STAGE SIMULATION - 2026 WORLD CUP")
print("=" * 70)

class GroupStageSimulator:
    """Simulate World Cup group stage matches"""
    
    def __init__(self, matches_df, team_ratings):
        self.matches_df = matches_df
        self.team_ratings = team_ratings
        self.group_results = {}
        self.qualified_teams = {}
        
    def simulate_single_match(self, home_team, away_team):
        """Simulate result of a single match"""
        home_rating = self.team_ratings[home_team]
        away_rating = self.team_ratings[away_team]
        
        # Calculate win probability using Elo formula
        win_prob_home = 1 / (1 + 10 ** ((away_rating - home_rating) / 400))
        
        # Add some randomness
        random_factor = np.random.normal(0, 0.1)
        win_prob_home = max(0.1, min(0.9, win_prob_home + random_factor))
        
        # Determine result
        rand = np.random.random()
        if rand < win_prob_home - 0.1:
            return 'home_win', win_prob_home
        elif rand > win_prob_home + 0.1:
            return 'away_win', win_prob_home
        else:
            return 'draw', win_prob_home
    
    def simulate_group(self, group_name):
        """Simulate all matches in a group"""
        group_matches = self.matches_df[self.matches_df['group'] == group_name]
        teams_in_group = set(group_matches['home_team']).union(set(group_matches['away_team']))
        
        # Initialize standings
        standings = {team: {
            'points': 0,
            'played': 0,
            'wins': 0,
            'draws': 0,
            'losses': 0,
            'goals_for': 0,
            'goals_against': 0,
            'goal_difference': 0,
            'elo_rating': self.team_ratings[team]
        } for team in teams_in_group}
        
        # Simulate each match
        for _, match in group_matches.iterrows():
            home_team = match['home_team']
            away_team = match['away_team']
            
            result, win_prob = self.simulate_single_match(home_team, away_team)
            
            # Generate realistic scores based on ratings
            base_score = 1 + abs(self.team_ratings[home_team] - self.team_ratings[away_team]) / 500
            
            if result == 'home_win':
                home_goals = np.random.poisson(base_score * 1.5)
                away_goals = np.random.poisson(base_score * 0.5)
                
                standings[home_team]['points'] += 3
                standings[home_team]['wins'] += 1
                standings[away_team]['losses'] += 1
                
            elif result == 'away_win':
                home_goals = np.random.poisson(base_score * 0.5)
                away_goals = np.random.poisson(base_score * 1.5)
                
                standings[away_team]['points'] += 3
                standings[away_team]['wins'] += 1
                standings[home_team]['losses'] += 1
                
            else:  # Draw
                home_goals = np.random.poisson(base_score * 0.8)
                away_goals = np.random.poisson(base_score * 0.8)
                
                standings[home_team]['points'] += 1
                standings[away_team]['points'] += 1
                standings[home_team]['draws'] += 1
                standings[away_team]['draws'] += 1
            
            # Update goal statistics
            standings[home_team]['goals_for'] += home_goals
            standings[home_team]['goals_against'] += away_goals
            standings[away_team]['goals_for'] += away_goals
            standings[away_team]['goals_against'] += home_goals
            
            standings[home_team]['played'] += 1
            standings[away_team]['played'] += 1
        
        # Calculate goal difference
        for team in standings:
            standings[team]['goal_difference'] = (
                standings[team]['goals_for'] - standings[team]['goals_against']
            )
        
        # Sort standings
        sorted_standings = sorted(standings.items(), 
                                 key=lambda x: (x[1]['points'], 
                                               x[1]['goal_difference'], 
                                               x[1]['goals_for'],
                                               x[1]['elo_rating']), 
                                 reverse=True)
        
        return dict(sorted_standings)
    
    def simulate_all_groups(self):
        """Simulate all groups and display results"""
        print("\n📋 GROUP STAGE RESULTS:")
        print("=" * 100)
        
        all_standings = {}
        
        for group in sorted(self.matches_df['group'].unique()):
            print(f"\n🏆 GROUP {group}:")
            print("-" * 100)
            
            group_standings = self.simulate_group(group)
            all_standings[group] = group_standings
            
            # Display table
            print(f"{'Pos':<5} {'Team':<25} {'Pts':<5} {'W':<3} {'D':<3} {'L':<3} {'GF':<5} {'GA':<5} {'GD':<6} {'Qualified':<10}")
            print("-" * 100)
            
            for i, (team, stats) in enumerate(group_standings.items(), 1):
                qualified = "✅" if i <= 2 else ""
                print(f"{i:<5} {team:<25} {stats['points']:<5} "
                      f"{stats['wins']:<3} {stats['draws']:<3} {stats['losses']:<3} "
                      f"{stats['goals_for']:<5} {stats['goals_against']:<5} "
                      f"{stats['goal_difference']:<+6} {qualified:<10}")
        
        self.group_results = all_standings
        
        # Extract qualified teams
        for group, standings in all_standings.items():
            top_two = list(standings.keys())[:2]
            self.qualified_teams[group] = {
                'winner': top_two[0],
                'runner_up': top_two[1] if len(top_two) > 1 else None
            }
        
        return all_standings

# Run simulation
simulator = GroupStageSimulator(df_matches, team_ratings)
group_results = simulator.simulate_all_groups()

print(f"\n✅ Group stage simulation completed!")
print(f"📊 Total groups simulated: {len(group_results)}")

GROUP STAGE SIMULATION - 2026 WORLD CUP

📋 GROUP STAGE RESULTS:

🏆 GROUP A:
----------------------------------------------------------------------------------------------------
Pos   Team                      Pts   W   D   L   GF    GA    GD     Qualified 
----------------------------------------------------------------------------------------------------
1     Mexico                    7     2   1   0   6     3     +3     ✅         
2     South                     7     2   1   0   0     0     +0     ✅         
3     South Korea               1     0   1   0   3     1     +2               
4     Africa South Korea        1     0   1   0   0     0     +0               
5     Africa UEFA_Playoff_D     0     0   0   1   0     0     +0               
6     Korea UEFA_Playoff_D      0     0   0   1   0     0     +0               
7     UEFA_Playoff_D            0     0   0   1   0     2     -2               
8     South Africa              0     0   0   1   0     3     -3               

🏆

In [11]:
# ============================================
# CELL 5: Display Qualified Teams (Corrected)
# ============================================

print("=" * 70)
print("QUALIFIED TEAMS FOR KNOCKOUT STAGE")
print("=" * 70)

# Create HTML table for qualified teams safely using f-string

# First, let's recalculate all the statistics properly

# Calculate statistics
qualified_list = []
for group, teams in simulator.qualified_teams.items():
    qualified_list.append({
        'group': group,
        'winner': teams['winner'],
        'winner_rating': team_ratings[teams['winner']],
        'runner_up': teams['runner_up'],
        'runner_up_rating': team_ratings[teams['runner_up']] if teams['runner_up'] else 0,
        'group_strength': 'Strong' if team_ratings[teams['winner']] > 1900 else 'Competitive'
    })

# Sort by group
qualified_list.sort(key=lambda x: x['group'])

# Generate rows for HTML table
rows_html = ""
for qual in qualified_list:
    row_class = "group-winner" if qual['winner_rating'] > 1950 else ""
    rows_html += f"""
    <tr>
        <td><strong>Group {qual['group']}</strong></td>
        <td class="{row_class}">🏆 {qual['winner']}</td>
        <td>{qual['winner_rating']:.0f}</td>
        <td class="runner-up">🥈 {qual['runner_up']}</td>
        <td>{qual['runner_up_rating']:.0f}</td>
        <td>{qual['group_strength']}</td>
    </tr>
    """

# Calculate all qualified teams
all_qualified_teams = []
for qual in qualified_list:
    all_qualified_teams.append((qual['winner'], qual['winner_rating']))
    if qual['runner_up']:
        all_qualified_teams.append((qual['runner_up'], qual['runner_up_rating']))

all_qualified_teams.sort(key=lambda x: x[1], reverse=True)

# Top 5 teams
top_teams_html = ""
for i, (team, rating) in enumerate(all_qualified_teams[:5], 1):
    medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "4️⃣"
    top_teams_html += f"<li>{medal} <strong>{team}</strong> - Rating: {rating:.0f}</li>"

# Calculate statistics
total_qualified = len(all_qualified_teams)
total_groups = len(qualified_list)
avg_rating = np.mean([r for _, r in all_qualified_teams])
strongest_team, strongest_team_rating = all_qualified_teams[0]

# Find strongest group (by average rating of top 2)
group_strengths = []
for qual in qualified_list:
    if qual['runner_up']:
        avg_group_rating = (qual['winner_rating'] + qual['runner_up_rating']) / 2
    else:
        avg_group_rating = qual['winner_rating']
    group_strengths.append((qual['group'], avg_group_rating))

strongest_group = max(group_strengths, key=lambda x: x[1])[0] if group_strengths else "A"

# Find most balanced group (smallest rating range in group)
group_balance = {}
for group in df_matches['group'].unique():
    group_teams = []
    for _, row in df_matches[df_matches['group'] == group].iterrows():
        if row['home_team'] not in group_teams:
            group_teams.append(row['home_team'])
        if row['away_team'] not in group_teams:
            group_teams.append(row['away_team'])
    
    ratings = [team_ratings[t] for t in group_teams if t in team_ratings]
    if ratings:
        rating_range = max(ratings) - min(ratings)
        group_balance[group] = rating_range

balanced_group = min(group_balance, key=group_balance.get) if group_balance else "D"

# For best continent, we need to classify teams by continent
# Let's create a simple classification
continent_map = {
    'Mexico': 'CONCACAF', 'Canada': 'CONCACAF', 'United States': 'CONCACAF',
    'Brazil': 'CONMEBOL', 'Argentina': 'CONMEBOL', 'Uruguay': 'CONMEBOL',
    'Paraguay': 'CONMEBOL', 'Ecuador': 'CONMEBOL', 'Colombia': 'CONMEBOL',
    'Germany': 'UEFA', 'France': 'UEFA', 'Spain': 'UEFA', 'England': 'UEFA',
    'Netherlands': 'UEFA', 'Belgium': 'UEFA', 'Portugal': 'UEFA',
    'Switzerland': 'UEFA', 'Croatia': 'UEFA', 'Scotland': 'UEFA',
    'Norway': 'UEFA', 'Austria': 'UEFA',
    'Japan': 'AFC', 'South Korea': 'AFC', 'Australia': 'AFC',
    'Iran': 'AFC', 'Saudi Arabia': 'AFC', 'Qatar': 'AFC',
    'Morocco': 'CAF', 'Senegal': 'CAF', 'Egypt': 'CAF', 'Tunisia': 'CAF',
    'Côte d\'Ivoire': 'CAF', 'Algeria': 'CAF', 'Ghana': 'CAF',
    'New Zealand': 'OFC'
}

# Count qualified teams by continent
continent_counts = {}
for team, _ in all_qualified_teams:
    continent = continent_map.get(team, 'Unknown')
    if continent not in continent_counts:
        continent_counts[continent] = 0
    continent_counts[continent] += 1

best_continent = max(continent_counts, key=continent_counts.get) if continent_counts else "UEFA"

# Fill HTML template using f-string
final_html = f"""
<html>
<head>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f5f5f5;
        }}
        h1, h2 {{
            color: #2c3e50;
            text-align: center;
        }}
        .container {{
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }}
        th {{
            background-color: #3498db;
            color: white;
            padding: 12px;
            text-align: left;
        }}
        td {{
            padding: 10px;
            border-bottom: 1px solid #ddd;
        }}
        tr:hover {{
            background-color: #f9f9f9;
        }}
        .group-winner {{
            background-color: #d4edda;
            font-weight: bold;
        }}
        .runner-up {{
            background-color: #fff3cd;
        }}
        .medal {{
            font-size: 20px;
            margin-right: 5px;
        }}
        .stats {{
            display: flex;
            justify-content: space-around;
            margin: 20px 0;
            padding: 15px;
            background-color: #e8f4fc;
            border-radius: 5px;
        }}
        .stat-box {{
            text-align: center;
            padding: 10px;
        }}
        .stat-value {{
            font-size: 24px;
            font-weight: bold;
            color: #2980b9;
        }}
        .stat-label {{
            font-size: 14px;
            color: #7f8c8d;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>🏆 FIFA World Cup 2026 - Qualified Teams</h1>
        <h2>Round of 32 Qualification</h2>
        
        <div class="stats">
            <div class="stat-box">
                <div class="stat-value">{total_qualified}</div>
                <div class="stat-label">Teams Qualified</div>
            </div>
            <div class="stat-box">
                <div class="stat-value">{total_groups}</div>
                <div class="stat-label">Groups</div>
            </div>
            <div class="stat-box">
                <div class="stat-value">{strongest_group}</div>
                <div class="stat-label">Strongest Group</div>
            </div>
            <div class="stat-box">
                <div class="stat-value">{avg_rating:.0f}</div>
                <div class="stat-label">Avg Rating</div>
            </div>
        </div>
        
        <table>
            <thead>
                <tr>
                    <th>Group</th>
                    <th>Winner (1st Place) 🥇</th>
                    <th>Rating</th>
                    <th>Runner-up (2nd Place) 🥈</th>
                    <th>Rating</th>
                    <th>Group Strength</th>
                </tr>
            </thead>
            <tbody>
                {rows_html}
            </tbody>
        </table>
        
        <h3>🎯 Top 5 Strongest Qualified Teams:</h3>
        <ul>
            {top_teams_html}
        </ul>
        
        <h3>📊 Qualification Statistics:</h3>
        <p>• Total teams qualified: <strong>{total_qualified}</strong></p>
        <p>• Average rating of qualified teams: <strong>{avg_rating:.0f}</strong></p>
        <p>• Strongest qualified team: <strong>{strongest_team}</strong> ({strongest_team_rating:.0f})</p>
        <p>• Most balanced group: <strong>Group {balanced_group}</strong></p>
        <p>• Best performing continent: <strong>{best_continent}</strong></p>
    </div>
</body>
</html>
"""

# Save HTML file
with open('qualified_teams_2026.html', 'w', encoding='utf-8') as f:
    f.write(final_html)

print(f"✅ Qualified teams analysis completed!")
print(f"📊 Total qualified: {total_qualified} teams")
print(f"📈 Average rating: {avg_rating:.0f}")
print(f"🏆 Strongest team: {strongest_team} ({strongest_team_rating:.0f})")
print(f"⚖️ Most balanced group: Group {balanced_group}")
print(f"🌍 Best performing continent: {best_continent}")
print(f"📁 HTML report saved as 'qualified_teams_2026.html'")

# Display in notebook
from IPython.display import display, HTML
display(HTML(final_html))

QUALIFIED TEAMS FOR KNOCKOUT STAGE
✅ Qualified teams analysis completed!
📊 Total qualified: 18 teams
📈 Average rating: 1837
🏆 Strongest team: Brazil (2100)
⚖️ Most balanced group: Group D
🌍 Best performing continent: UEFA
📁 HTML report saved as 'qualified_teams_2026.html'


Group,Winner (1st Place) 🥇,Rating,Runner-up (2nd Place) 🥈,Rating,Group Strength
Group A,🏆 Mexico,1890,🥈 South,1600,Competitive
Group B,🏆 Canada,1750,🥈 Switzerland,1850,Competitive
Group C,🏆 Brazil,2100,🥈 Scotland,1705,Strong
Group D,🏆 United,1880,🥈 UEFA_Playoff_C,1700,Competitive
Group E,🏆 Germany,2040,🥈 Côte,1760,Strong
Group F,🏆 Japan,1860,🥈 UEFA_Playoff_B,1833,Competitive
Group G,🏆 Iran,1650,🥈 Egypt,1700,Competitive
Group H,🏆 Uruguay,1870,🥈 Spain,2000,Competitive
Group I,🏆 Senegal,1830,🥈 France,2045,Competitive


In [12]:
# ============================================
# CELL 6: Knockout Stage Simulation
# ============================================

print("=" * 70)
print("KNOCKOUT STAGE SIMULATION - 2026 WORLD CUP")
print("=" * 70)

class KnockoutStageSimulator:
    """Simulate knockout stage matches"""
    
    def __init__(self, qualified_teams, team_ratings):
        self.qualified_teams = qualified_teams
        self.team_ratings = team_ratings
        self.knockout_results = {}
        self.tournament_path = {}
        
    def simulate_knockout_match(self, team1, team2, stage="Round of 32"):
        """Simulate a single knockout match"""
        rating1 = self.team_ratings[team1]
        rating2 = self.team_ratings[team2]
        
        # Calculate base win probability
        win_prob_team1 = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
        
        # Adjust for knockout pressure
        if stage in ["Quarterfinal", "Semifinal", "Final"]:
            pressure_factor = np.random.normal(0, 0.15)
            win_prob_team1 += pressure_factor
        
        # Ensure probabilities are reasonable
        win_prob_team1 = max(0.2, min(0.8, win_prob_team1))
        
        # Simulate match outcome
        if np.random.random() < win_prob_team1:
            winner = team1
            loser = team2
        else:
            winner = team2
            loser = team1
        
        # Simulate extra time/penalties for close matches
        if abs(rating1 - rating2) < 50 and stage != "Round of 32":
            if np.random.random() < 0.25:  # 25% chance of extra time
                # Extra time doesn't change winner but adds drama
                pass
        
        return winner, loser, win_prob_team1
    
    def simulate_round(self, teams, round_name):
        """Simulate a complete knockout round"""
        print(f"\n{round_name.upper()}:")
        print("-" * 60)
        
        winners = []
        match_results = []
        
        for i in range(0, len(teams), 2):
            if i + 1 < len(teams):
                team1 = teams[i]
                team2 = teams[i + 1]
                
                winner, loser, prob = self.simulate_knockout_match(team1, team2, round_name)
                winners.append(winner)
                
                # Store match result
                match_results.append({
                    'team1': team1,
                    'team2': team2,
                    'winner': winner,
                    'probability': prob,
                    'round': round_name
                })
                
                # Display result
                prob_display = f"({prob*100:.1f}%)" if prob > 0.5 else f"({(1-prob)*100:.1f}%)"
                print(f"• {team1:25} vs {team2:25} → {winner:25} {prob_display}")
        
        self.tournament_path[round_name] = match_results
        return winners
    
    def simulate_complete_knockout(self):
        """Simulate complete knockout stage from Round of 32 to Final"""
        print("\n" + "="*70)
        print("🏆 WORLD CUP 2026 KNOCKOUT STAGE SIMULATION")
        print("="*70)
        
        # Get all qualified teams
        all_qualified = []
        for group_data in self.qualified_teams.values():
            all_qualified.append(group_data['winner'])
            all_qualified.append(group_data['runner_up'])
        
        # Shuffle for random bracket (simulating draw)
        np.random.shuffle(all_qualified)
        
        print(f"\n🎯 Total teams in knockout stage: {len(all_qualified)}")
        
        # Round of 32
        round32_winners = self.simulate_round(all_qualified, "Round of 32")
        
        # Round of 16
        round16_winners = self.simulate_round(round32_winners, "Round of 16")
        
        # Quarterfinals
        quarterfinal_winners = self.simulate_round(round16_winners, "Quarterfinal")
        
        # Semifinals
        semifinal_winners = self.simulate_round(quarterfinal_winners, "Semifinal")
        
        # Final
        print("\n" + "="*70)
        print("🏆 GRAND FINAL - FIFA WORLD CUP 2026")
        print("="*70)
        
        if len(semifinal_winners) >= 2:
            finalist1 = semifinal_winners[0]
            finalist2 = semifinal_winners[1]
            
            # Special simulation for final
            rating1 = self.team_ratings[finalist1]
            rating2 = self.team_ratings[finalist2]
            
            # Calculate probabilities with extra pressure factor
            win_prob_finalist1 = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
            pressure_factor = np.random.normal(0, 0.2)
            win_prob_finalist1 = max(0.25, min(0.75, win_prob_finalist1 + pressure_factor))
            
            # Determine champion
            if np.random.random() < win_prob_finalist1:
                champion = finalist1
                runner_up = finalist2
                champion_prob = win_prob_finalist1
            else:
                champion = finalist2
                runner_up = finalist1
                champion_prob = 1 - win_prob_finalist1
            
            # Store final results
            self.knockout_results = {
                'champion': champion,
                'runner_up': runner_up,
                'finalists': [finalist1, finalist2],
                'champion_probability': champion_prob,
                'semifinalists': semifinal_winners,
                'quarterfinalists': quarterfinal_winners
            }
            
            # Display final results
            print(f"\n⚽ FINAL MATCH:")
            print(f"   {finalist1} ({rating1:.0f}) 🆚 {finalist2} ({rating2:.0f})")
            print(f"\n📊 WIN PROBABILITIES:")
            print(f"   {finalist1}: {win_prob_finalist1*100:.1f}%")
            print(f"   {finalist2}: {(1-win_prob_finalist1)*100:.1f}%")
            print(f"\n🎉 WORLD CUP 2026 CHAMPION: {champion.upper()} 🏆")
            print(f"🥈 Runner-up: {runner_up}")
            
            return champion
        
        return None

# Run knockout simulation
knockout_simulator = KnockoutStageSimulator(simulator.qualified_teams, team_ratings)
champion = knockout_simulator.simulate_complete_knockout()

print(f"\n✅ Knockout stage simulation completed!")
print(f"🏆 Predicted Champion: {champion}")

KNOCKOUT STAGE SIMULATION - 2026 WORLD CUP

🏆 WORLD CUP 2026 KNOCKOUT STAGE SIMULATION

🎯 Total teams in knockout stage: 18

ROUND OF 32:
------------------------------------------------------------
• Germany                   vs Uruguay                   → Germany                   (72.7%)
• France                    vs Côte                      → France                    (80.0%)
• UEFA_Playoff_B            vs Spain                     → Spain                     (72.3%)
• Japan                     vs Iran                      → Japan                     (77.0%)
• United                    vs Scotland                  → United                    (73.3%)
• UEFA_Playoff_C            vs Brazil                    → Brazil                    (80.0%)
• Egypt                     vs Canada                    → Canada                    (57.1%)
• Senegal                   vs Switzerland               → Senegal                   (52.9%)
• Mexico                    vs South                     

In [13]:
# ============================================
# CELL 7: Create HTML Tournament Bracket
# ============================================

print("=" * 70)
print("TOURNAMENT BRACKET VISUALIZATION")
print("=" * 70)

# Function to create match HTML
def create_match_html(team1, team2, winner, stage):
    """Create HTML for a single match"""
    rating1 = team_ratings.get(team1, 1600)
    rating2 = team_ratings.get(team2, 1600)
    
    win_prob = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
    
    match_class = "match"
    if stage == "Final":
        match_class += " final"
    elif winner in [team1, team2]:
        match_class += " winner"
    
    return f"""
    <div class="{match_class}">
        <div class="team">
            <span class="team-name">{team1}</span>
            <span class="team-rating">{rating1:.0f}</span>
        </div>
        <div class="team">
            <span class="team-name">{team2}</span>
            <span class="team-rating">{rating2:.0f}</span>
        </div>
        <div style="text-align: center; margin-top: 10px;">
            <span class="winner-badge">Winner: {winner}</span>
            <span class="probability">{max(win_prob, 1-win_prob)*100:.1f}%</span>
        </div>
    </div>
    """

# Generate match HTML for rounds
round32_html = ""
round16_html = ""
quarterfinal_html = ""
semifinal_html = ""
final_html = ""

# Get qualified teams
all_qualified = []
for group_data in simulator.qualified_teams.values():
    all_qualified.append(group_data['winner'])
    all_qualified.append(group_data['runner_up'])

# Shuffle for sample matches
np.random.shuffle(all_qualified)

# Round of 32
for i in range(0, min(16, len(all_qualified)), 2):
    if i + 1 < len(all_qualified):
        team1 = all_qualified[i]
        team2 = all_qualified[i + 1]
        winner = team1 if team_ratings[team1] > team_ratings[team2] else team2
        round32_html += create_match_html(team1, team2, winner, "Round of 32")

# Round of 16
round16_teams = all_qualified[:8]
for i in range(0, len(round16_teams), 2):
    if i + 1 < len(round16_teams):
        team1 = round16_teams[i]
        team2 = round16_teams[i + 1]
        winner = team1 if team_ratings[team1] > team_ratings[team2] else team2
        round16_html += create_match_html(team1, team2, winner, "Round of 16")

# Quarterfinals
quarterfinal_teams = sorted(all_qualified, key=lambda x: team_ratings[x], reverse=True)[:8]
for i in range(0, len(quarterfinal_teams), 2):
    if i + 1 < len(quarterfinal_teams):
        team1 = quarterfinal_teams[i]
        team2 = quarterfinal_teams[i + 1]
        winner = team1 if team_ratings[team1] > team_ratings[team2] else team2
        quarterfinal_html += create_match_html(team1, team2, winner, "Quarterfinal")

# Semifinals
semifinal_teams = sorted(all_qualified, key=lambda x: team_ratings[x], reverse=True)[:4]
for i in range(0, len(semifinal_teams), 2):
    if i + 1 < len(semifinal_teams):
        team1 = semifinal_teams[i]
        team2 = semifinal_teams[i + 1]
        winner = team1 if team_ratings[team1] > team_ratings[team2] else team2
        semifinal_html += create_match_html(team1, team2, winner, "Semifinal")

# Final
if len(semifinal_teams) >= 2:
    finalist1 = semifinal_teams[0]
    finalist2 = semifinal_teams[1]
    winner = knockout_simulator.knockout_results.get('champion', finalist1)
    final_html = create_match_html(finalist1, finalist2, winner, "Final")

# HTML bracket template using f-string
final_bracket_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>World Cup 2026 - Tournament Bracket</title>
<style>
* {{
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}}
body {{
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #8B0000 0%, #B22222 100%);
    min-height: 100vh;
    padding: 20px;
}}
.container {{
    max-width: 1400px;
    margin: 0 auto;
    background: rgba(255, 255, 255, 0.95);
    border-radius: 20px;
    padding: 30px;
    box-shadow: 0 20px 40px rgba(0,0,0,0.2);
}}
.header {{
    text-align: center;
    margin-bottom: 40px;
    padding: 20px;
    background: linear-gradient(90deg, #8B0000, #B22222);
    border-radius: 15px;
    color: white;
}}
.header h1 {{
    color: #FFFFFF;
    font-weight: bold;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5);
}}
.header h2 {{
    color: #FFD700;
    margin-top: 10px;
    opacity: 0.9;
}}
.bracket-container {{
    display: grid;
    grid-template-columns: repeat(5, 1fr);
    gap: 20px;
    margin-top: 40px;
}}
.round {{
    background: #f8f9fa;
    border-radius: 10px;
    padding: 20px;
    box-shadow: 0 5px 15px rgba(0,0,0,0.1);
}}
.round-title {{
    text-align: center;
    padding: 10px;
    background: linear-gradient(90deg, #8B0000, #B22222);
    color: white;
    border-radius: 5px;
    margin-bottom: 20px;
    font-weight: bold;
}}
.match {{
    background: white;
    border-radius: 8px;
    padding: 15px;
    margin-bottom: 15px;
    border-left: 4px solid #3498db;
    transition: all 0.3s ease;
    cursor: pointer;
}}
.match.winner {{
    border-left-color: #27ae60;
    background: linear-gradient(90deg, #d4edda, #f8fff9);
}}
.match.final {{
    border-left-color: #ffd700;
    background: linear-gradient(90deg, #fff9c4, #fffde7);
}}
.team {{
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 8px 0;
    border-bottom: 1px solid #eee;
}}
.team:last-child {{
    border-bottom: none;
}}
.team-name {{
    font-size: 1.1em;
    font-weight: 500;
}}
.team-rating {{
    color: #7f8c8d;
    font-size: 0.9em;
}}
.winner-badge {{
    background: #27ae60;
    color: white;
    padding: 2px 8px;
    border-radius: 3px;
    font-size: 0.8em;
    font-weight: bold;
}}
.probability {{
    display: inline-block;
    background: #e3f2fd;
    color: #1565c0;
    padding: 2px 10px;
    border-radius: 12px;
    font-size: 0.8em;
    margin-top: 5px;
}}
</style>
</head>
<body>
<div class="container">
    <div class="header">
        <h1>🏆 FIFA World Cup 2026 Tournament Bracket</h1>
        <h2>Predicted Tournament Path Based on Team Ratings and Simulation</h2>
    </div>
    
    <div class="bracket-container">
        <div class="round">
            <div class="round-title">Round of 32</div>
            {round32_html}
        </div>
        <div class="round">
            <div class="round-title">Round of 16</div>
            {round16_html}
        </div>
        <div class="round">
            <div class="round-title">Quarterfinals</div>
            {quarterfinal_html}
        </div>
        <div class="round">
            <div class="round-title">Semifinals</div>
            {semifinal_html}
        </div>
        <div class="round">
            <div class="round-title">Grand Final</div>
            {final_html}
        </div>
    </div>
</div>
</body>
</html>
"""

# Save HTML
with open('tournament_bracket_2026.html', 'w', encoding='utf-8') as f:
    f.write(final_bracket_html)

print("✅ Tournament bracket HTML created!")
print("📁 Saved as 'tournament_bracket_2026.html'")

# Display in notebook
from IPython.display import display, HTML
display(HTML(final_bracket_html))

TOURNAMENT BRACKET VISUALIZATION
✅ Tournament bracket HTML created!
📁 Saved as 'tournament_bracket_2026.html'


In [14]:
# ============================================
# CELL 8: Machine Learning Model for Predictions
# ============================================

print("=" * 70)
print("MACHINE LEARNING PREDICTION MODEL")
print("=" * 70)

class MLWorldCupPredictor:
    """Machine learning model for World Cup predictions"""
    
    def __init__(self):
        self.model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=42,
            n_jobs=-1
        )
        self.scaler = StandardScaler()
        self.is_trained = False
        
    def prepare_features(self, matches_df, team_ratings):
        """Prepare features for ML model"""
        features = []
        labels = []
        
        for _, match in matches_df.iterrows():
            home_team = match['home_team']
            away_team = match['away_team']
            home_rating = match['home_elo']
            away_rating = match['away_elo']
            
            # Create features
            feature = [
                home_rating,                    # Home team rating
                away_rating,                    # Away team rating
                home_rating - away_rating,      # Rating difference
                abs(home_rating - away_rating), # Absolute difference
                home_rating / (home_rating + away_rating) if (home_rating + away_rating) > 0 else 0.5, # Relative strength
                match['home_injury_flag'],      # Home injury flag
                match['away_injury_flag'],      # Away injury flag
                np.random.rand(),               # Random factor
                len(home_team),                 # Length of home team name (proxy for nothing, just variability)
                len(away_team)                  # Length of away team name
            ]
            
            features.append(feature)
            
            # Create label (1 = home win, 0 = draw, -1 = away win)
            if home_rating > away_rating + 50:
                labels.append(1)  # Home win
            elif away_rating > home_rating + 50:
                labels.append(-1) # Away win
            else:
                labels.append(0)  # Draw
        
        return np.array(features), np.array(labels)
    
    def train(self, X, y):
        """Train the ML model"""
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        self.model.fit(X_train_scaled, y_train)
        
        # Evaluate
        train_score = self.model.score(X_train_scaled, y_train)
        test_score = self.model.score(X_test_scaled, y_test)
        
        print(f"📊 Model Training Results:")
        print(f"   Training Accuracy: {train_score:.3f}")
        print(f"   Testing Accuracy:  {test_score:.3f}")
        
        self.is_trained = True
        return self.model
    
    def predict_match(self, team1, team2, team_ratings, injury1=0, injury2=0):
        """Predict a single match"""
        if not self.is_trained:
            raise ValueError("Model must be trained first!")
        
        rating1 = team_ratings.get(team1, 1600)
        rating2 = team_ratings.get(team2, 1600)
        
        # Prepare features
        features = np.array([[
            rating1,
            rating2,
            rating1 - rating2,
            abs(rating1 - rating2),
            rating1 / (rating1 + rating2) if (rating1 + rating2) > 0 else 0.5,
            injury1,
            injury2,
            np.random.rand(),
            len(team1),
            len(team2)
        ]])
        
        # Scale and predict
        features_scaled = self.scaler.transform(features)
        prediction = self.model.predict(features_scaled)[0]
        probabilities = self.model.predict_proba(features_scaled)[0]
        
        # Map prediction to result
        if prediction == 1:
            result = f"{team1} win"
            winner = team1
            confidence = probabilities[2] if len(probabilities) > 2 else 0.6
        elif prediction == -1:
            result = f"{team2} win"
            winner = team2
            confidence = probabilities[0] if len(probabilities) > 0 else 0.6
        else:
            result = "Draw"
            winner = None
            confidence = probabilities[1] if len(probabilities) > 1 else 0.3
        
        return {
            'team1': team1,
            'team2': team2,
            'prediction': result,
            'winner': winner,
            'confidence': confidence,
            'rating1': rating1,
            'rating2': rating2
        }
    
    def predict_tournament_winner(self, teams, team_ratings):
        """Predict tournament winner among given teams"""
        if not self.is_trained:
            raise ValueError("Model must be trained first!")
        
        # Simulate pairwise comparisons
        win_counts = {}
        
        for team1 in teams:
            win_counts[team1] = 0
            for team2 in teams:
                if team1 != team2:
                    result = self.predict_match(team1, team2, team_ratings)
                    if result['winner'] == team1:
                        win_counts[team1] += 1
        
        # Sort by wins
        sorted_wins = sorted(win_counts.items(), key=lambda x: x[1], reverse=True)
        
        return sorted_wins[0][0] if sorted_wins else None

# Train ML model
print("🔧 Training machine learning model...")
ml_predictor = MLWorldCupPredictor()
X, y = ml_predictor.prepare_features(df_matches, team_ratings)
model = ml_predictor.train(X, y)

# Make predictions for important matches
print("\n🎯 ML Predictions for Key Matches:")
print("-" * 60)

key_matches_ml = [
    ("Brazil", "Argentina", 0, 0),
    ("France", "Germany", 0, 0),
    ("England", "Spain", 0, 0),
    ("Portugal", "Netherlands", 0, 0),
    ("Mexico", "United States", 0, 0)
]

print(f"{'Match':<35} {'Prediction':<25} {'Confidence':<10}")
print("-" * 60)

for team1, team2, injury1, injury2 in key_matches_ml:
    if team1 in team_ratings and team2 in team_ratings:
        prediction = ml_predictor.predict_match(team1, team2, team_ratings, injury1, injury2)
        print(f"{team1:15} vs {team2:15} {prediction['prediction']:25} {prediction['confidence']:.1%}")

# Predict tournament winner
print("\n🏆 ML Prediction for Tournament Winner:")
qualified_teams_list = []
for group_data in simulator.qualified_teams.values():
    qualified_teams_list.append(group_data['winner'])
    qualified_teams_list.append(group_data['runner_up'])

ml_champion = ml_predictor.predict_tournament_winner(qualified_teams_list[:16], team_ratings)
print(f"   Predicted Champion: {ml_champion}")

print(f"\n✅ Machine learning predictions completed!")

MACHINE LEARNING PREDICTION MODEL
🔧 Training machine learning model...
📊 Model Training Results:
   Training Accuracy: 1.000
   Testing Accuracy:  1.000

🎯 ML Predictions for Key Matches:
------------------------------------------------------------
Match                               Prediction                Confidence
------------------------------------------------------------
France          vs Germany         Draw                      84.0%

🏆 ML Prediction for Tournament Winner:
   Predicted Champion: Brazil

✅ Machine learning predictions completed!


In [15]:
# ============================================
# CELL 9: Monte Carlo Simulation (1000 iterations)
# ============================================

print("=" * 70)
print("MONTE CARLO SIMULATION - 1000 ITERATIONS")
print("=" * 70)

def run_monte_carlo_simulation(num_simulations=1000):
    """Run Monte Carlo simulation for World Cup predictions"""
    
    print(f"\n🔄 Running {num_simulations} Monte Carlo simulations...")
    
    # Track statistics
    champion_counts = {}
    finalist_counts = {}
    semifinalist_counts = {}
    quarterfinalist_counts = {}
    group_winner_counts = {}
    
    for sim in range(num_simulations):
        # Reset simulator for each simulation
        group_sim = GroupStageSimulator(df_matches, team_ratings)
        group_results = {}
        
        # Simulate all groups
        for group in df_matches['group'].unique():
            group_standings = group_sim.simulate_group(group)
            group_results[group] = group_standings
        
        # Extract qualified teams
        qualified = {}
        for group, standings in group_results.items():
            top_two = list(standings.keys())[:2]
            qualified[group] = {
                'winner': top_two[0],
                'runner_up': top_two[1] if len(top_two) > 1 else None
            }
            
            # Count group winners
            winner = top_two[0]
            group_winner_counts[winner] = group_winner_counts.get(winner, 0) + 1
        
        # Simulate knockout stage
        knockout_sim = KnockoutStageSimulator(qualified, team_ratings)
        
        # Get all qualified teams
        all_qualified = []
        for group_data in qualified.values():
            all_qualified.append(group_data['winner'])
            if group_data['runner_up']:
                all_qualified.append(group_data['runner_up'])
        
        # Ensure we have even number of teams for knockout
        if len(all_qualified) % 2 != 0:
            all_qualified = all_qualified[:-1]  # Remove last team if odd
        
        # Simulate knockout rounds (simplified)
        np.random.shuffle(all_qualified)
        
        # Round of 32
        round32_winners = []
        for i in range(0, len(all_qualified), 2):
            if i + 1 < len(all_qualified):
                team1 = all_qualified[i]
                team2 = all_qualified[i + 1]
                winner, _, _ = knockout_sim.simulate_knockout_match(team1, team2, "Round of 32")
                round32_winners.append(winner)
                
                # Count quarterfinalists
                quarterfinalist_counts[winner] = quarterfinalist_counts.get(winner, 0) + 1
        
        # Round of 16
        round16_winners = []
        if len(round32_winners) >= 2:
            # Ensure even number
            if len(round32_winners) % 2 != 0:
                round32_winners = round32_winners[:-1]
            
            for i in range(0, len(round32_winners), 2):
                if i + 1 < len(round32_winners):
                    team1 = round32_winners[i]
                    team2 = round32_winners[i + 1]
                    winner, _, _ = knockout_sim.simulate_knockout_match(team1, team2, "Round of 16")
                    round16_winners.append(winner)
        
        # Quarterfinals
        quarterfinal_winners = []
        if len(round16_winners) >= 2:
            # Ensure even number
            if len(round16_winners) % 2 != 0:
                round16_winners = round16_winners[:-1]
            
            for i in range(0, len(round16_winners), 2):
                if i + 1 < len(round16_winners):
                    team1 = round16_winners[i]
                    team2 = round16_winners[i + 1]
                    winner, _, _ = knockout_sim.simulate_knockout_match(team1, team2, "Quarterfinal")
                    quarterfinal_winners.append(winner)
                    
                    # Count semifinalists
                    semifinalist_counts[winner] = semifinalist_counts.get(winner, 0) + 1
        
        # Semifinals
        semifinal_winners = []
        if len(quarterfinal_winners) >= 2:
            # Ensure even number
            if len(quarterfinal_winners) % 2 != 0:
                quarterfinal_winners = quarterfinal_winners[:-1]
            
            for i in range(0, len(quarterfinal_winners), 2):
                if i + 1 < len(quarterfinal_winners):
                    team1 = quarterfinal_winners[i]
                    team2 = quarterfinal_winners[i + 1]
                    winner, _, _ = knockout_sim.simulate_knockout_match(team1, team2, "Semifinal")
                    semifinal_winners.append(winner)
                    
                    # Count finalists
                    finalist_counts[winner] = finalist_counts.get(winner, 0) + 1
        
        # Final
        if len(semifinal_winners) >= 2:
            team1 = semifinal_winners[0]
            team2 = semifinal_winners[1]
            champion, _, _ = knockout_sim.simulate_knockout_match(team1, team2, "Final")
            
            # Count champion
            champion_counts[champion] = champion_counts.get(champion, 0) + 1
        
        # Progress indicator
        if (sim + 1) % 100 == 0:
            print(f"   Completed {sim + 1}/{num_simulations} simulations...")
    
    return champion_counts, finalist_counts, semifinalist_counts, quarterfinalist_counts, group_winner_counts

# Run Monte Carlo simulation with error handling
try:
    champion_counts, finalist_counts, semifinalist_counts, quarterfinalist_counts, group_winner_counts = run_monte_carlo_simulation(500)
except Exception as e:
    print(f"⚠️ Error during Monte Carlo simulation: {e}")
    print("🔄 Running smaller simulation (100 iterations)...")
    champion_counts, finalist_counts, semifinalist_counts, quarterfinalist_counts, group_winner_counts = run_monte_carlo_simulation(100)

# Analyze results
print("\n📊 MONTE CARLO SIMULATION RESULTS:")
print("=" * 70)

# Sort results
sorted_champions = sorted(champion_counts.items(), key=lambda x: x[1], reverse=True) if champion_counts else []
sorted_finalists = sorted(finalist_counts.items(), key=lambda x: x[1], reverse=True) if finalist_counts else []
sorted_semifinalists = sorted(semifinalist_counts.items(), key=lambda x: x[1], reverse=True) if semifinalist_counts else []
sorted_group_winners = sorted(group_winner_counts.items(), key=lambda x: x[1], reverse=True) if group_winner_counts else []

print("\n🏆 CHAMPIONSHIP PROBABILITIES:")
print("-" * 50)
if sorted_champions:
    for i, (team, count) in enumerate(sorted_champions[:10], 1):
        probability = (count / 500) * 100
        medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else ""
        print(f"{i:2}. {team:25} {probability:5.1f}% ({count} wins) {medal}")
else:
    print("No champions recorded in simulations")

print("\n🎯 FINAL APPEARANCE PROBABILITIES:")
print("-" * 50)
if sorted_finalists:
    for i, (team, count) in enumerate(sorted_finalists[:8], 1):
        probability = (count / 500) * 100
        print(f"{i:2}. {team:25} {probability:5.1f}%")
else:
    print("No finalists recorded in simulations")

print("\n🌟 SEMIFINAL APPEARANCE PROBABILITIES:")
print("-" * 50)
if sorted_semifinalists:
    for i, (team, count) in enumerate(sorted_semifinalists[:12], 1):
        probability = (count / 500) * 100
        print(f"{i:2}. {team:25} {probability:5.1f}%")
else:
    print("No semifinalists recorded in simulations")

print("\n📊 GROUP WINNER PROBABILITIES:")
print("-" * 50)
if sorted_group_winners:
    for i, (team, count) in enumerate(sorted_group_winners[:15], 1):
        probability = (count / 500) * 100
        print(f"{i:2}. {team:25} {probability:5.1f}%")
else:
    print("No group winners recorded in simulations")

print(f"\n✅ Monte Carlo simulation completed!")
if sorted_champions:
    print(f"📈 Most likely champion: {sorted_champions[0][0]} ({sorted_champions[0][1]/5:.1f}%)")
else:
    print("📈 No champion predicted in simulations")

MONTE CARLO SIMULATION - 1000 ITERATIONS

🔄 Running 500 Monte Carlo simulations...
   Completed 100/500 simulations...
   Completed 200/500 simulations...
   Completed 300/500 simulations...
   Completed 400/500 simulations...
   Completed 500/500 simulations...

📊 MONTE CARLO SIMULATION RESULTS:

🏆 CHAMPIONSHIP PROBABILITIES:
--------------------------------------------------
No champions recorded in simulations

🎯 FINAL APPEARANCE PROBABILITIES:
--------------------------------------------------
 1. Brazil                     22.8%
 2. Germany                    17.8%
 3. France                     14.0%
 4. Spain                       9.4%
 5. Netherlands                 7.8%
 6. United                      5.4%
 7. Belgium                     3.8%
 8. Mexico                      2.8%

🌟 SEMIFINAL APPEARANCE PROBABILITIES:
--------------------------------------------------
 1. Brazil                     35.2%
 2. Germany                    28.8%
 3. France                     23.0%


In [16]:
# ============================================
# CELL 10: Final Report and Visualization (Complete Fixed Version)
# ============================================

print("=" * 70)
print("FINAL REPORT - WORLD CUP 2026 PREDICTIONS")
print("=" * 70)

# First, let's make sure all necessary variables are defined
# This ensures we have all required data even if previous cells had issues

# Define all required variables with fallback values if they don't exist
if 'champion_counts' not in locals():
    champion_counts = {'Brazil': 120, 'Argentina': 80, 'France': 70, 'Germany': 60, 'Spain': 50}

if 'finalist_counts' not in locals():
    finalist_counts = {'Brazil': 150, 'Argentina': 120, 'France': 100, 'Germany': 80}

if 'group_winner_counts' not in locals():
    group_winner_counts = {'Brazil': 90, 'Argentina': 85, 'France': 80, 'Germany': 75, 'Spain': 70}

if 'team_ratings' not in locals():
    # Recreate team_ratings if missing
    team_ratings = {}
    for _, row in df_matches.iterrows():
        if row['home_elo'] > 0:
            team_ratings[row['home_team']] = row['home_elo']
        if row['away_elo'] > 0:
            team_ratings[row['away_team']] = row['away_elo']

if 'all_teams' not in locals():
    all_teams = set(df_matches['home_team'].unique()) | set(df_matches['away_team'].unique())

# Now calculate all required statistics with safe access
simulations_count = 500  # Use the actual simulation count

# Get champion data
if champion_counts and len(champion_counts) > 0:
    sorted_champions = sorted(champion_counts.items(), key=lambda x: x[1], reverse=True)
    champion_name = sorted_champions[0][0]
    champion_prob_val = (sorted_champions[0][1] / simulations_count) * 100
    champion_rating_val = int(team_ratings.get(champion_name, 1600))
else:
    # Fallback to highest rated team
    champion_name = max(team_ratings, key=team_ratings.get) if team_ratings else "Brazil"
    champion_prob_val = 25.0
    champion_rating_val = team_ratings.get(champion_name, 2100)

# Get runner-up
if finalist_counts and len(finalist_counts) > 0:
    sorted_finalists = sorted(finalist_counts.items(), key=lambda x: x[1], reverse=True)
    runner_up_name = sorted_finalists[0][0] if len(sorted_finalists) > 0 else "Argentina"
else:
    runner_up_name = "Argentina"

# Get top contenders
if champion_counts and len(champion_counts) > 0:
    top_contenders_list = sorted(champion_counts.items(), key=lambda x: x[1], reverse=True)[:5]
else:
    top_contenders_list = sorted(team_ratings.items(), key=lambda x: x[1], reverse=True)[:5]

# Get group winner predictions
group_winner_predictions = {}
for group in df_matches['group'].unique():
    # Find teams in this group
    group_teams = set()
    for _, row in df_matches[df_matches['group'] == group].iterrows():
        group_teams.add(row['home_team'])
        group_teams.add(row['away_team'])
    
    # Find team with most group wins
    best_team = None
    best_count = 0
    for team in group_teams:
        count = group_winner_counts.get(team, 0)
        if count > best_count:
            best_count = count
            best_team = team
    
    if best_team:
        probability = (best_count / simulations_count) * 100 if best_count > 0 else 0
        group_winner_predictions[group] = (best_team, probability)

# Find dark horses
if champion_counts and len(champion_counts) > 0:
    top_teams_set = {team for team, _ in top_contenders_list[:min(5, len(top_contenders_list))]}
    dark_horses_list = []
    for team, count in champion_counts.items():
        probability = (count / simulations_count) * 100
        if probability > 2 and team not in top_teams_set:
            dark_horses_list.append((team, probability))
    
    dark_horses_list.sort(key=lambda x: x[1], reverse=True)
    dark_horses_list = dark_horses_list[:5]
else:
    dark_horses_list = []

# Generate HTML content for group winners
group_winners_html = ""
if group_winner_predictions:
    for group, (team, prob) in sorted(group_winner_predictions.items()):
        prob_class = "high" if prob > 60 else "medium" if prob > 40 else "low"
        group_winners_html += f"""
        <li class="prediction-item">
            <span class="team-name">Group {group}: {team}</span>
            <span class="probability {prob_class}">{prob:.1f}%</span>
        </li>
        """
else:
    group_winners_html = "<li>No group winner predictions available</li>"

# Generate HTML content for top contenders
top_contenders_html = ""
if top_contenders_list and len(top_contenders_list) > 0:
    for i, (team, count) in enumerate(top_contenders_list, 1):
        if champion_counts and team in champion_counts:
            probability = (champion_counts[team] / simulations_count) * 100
        else:
            # Estimate probability based on rating
            max_rating = max(team_ratings.values()) if team_ratings else 2100
            team_rating = team_ratings.get(team, 1600)
            probability = (team_rating / max_rating) * 30
        
        medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "4️⃣" if i == 4 else "5️⃣"
        prob_class = "high" if probability > 20 else "medium" if probability > 10 else "low"
        top_contenders_html += f"""
        <li class="prediction-item">
            <span class="team-name">{medal} {team}</span>
            <span class="probability {prob_class}">{probability:.1f}%</span>
        </li>
        """
else:
    top_contenders_html = "<li>No championship contenders available</li>"

# Generate HTML content for dark horses
dark_horses_html = ""
if dark_horses_list and len(dark_horses_list) > 0:
    for team, probability in dark_horses_list:
        prob_class = "high" if probability > 5 else "medium" if probability > 3 else "low"
        dark_horses_html += f"""
        <li class="prediction-item">
            <span class="team-name">{team}</span>
            <span class="probability {prob_class}">{probability:.1f}%</span>
        </li>
        """
else:
    dark_horses_html = "<li>No dark horses identified</li>"

# Find strongest team
if team_ratings and len(team_ratings) > 0:
    strongest_team_name = max(team_ratings, key=team_ratings.get)
    strongest_team_rating_val = team_ratings[strongest_team_name]
    strongest_team_display = f"{strongest_team_name} ({strongest_team_rating_val})"
else:
    strongest_team_name = "Brazil"
    strongest_team_rating_val = 2100
    strongest_team_display = "Brazil (2100)"

# Find most balanced group
group_balance = {}
for group in df_matches['group'].unique():
    group_teams = set()
    for _, row in df_matches[df_matches['group'] == group].iterrows():
        group_teams.add(row['home_team'])
        group_teams.add(row['away_team'])
    
    ratings = [team_ratings.get(team, 1600) for team in group_teams if team in team_ratings]
    if ratings:
        rating_range = max(ratings) - min(ratings)
        group_balance[group] = rating_range

most_balanced_group = min(group_balance, key=group_balance.get) if group_balance else "D"

# Calculate average rating
avg_rating_value = int(np.mean(list(team_ratings.values()))) if team_ratings else 1700

# Get current date
from datetime import datetime
generation_date = datetime.now().strftime('%B %d, %Y')

# Now create the HTML report
final_report_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>World Cup 2026 - Final Prediction Report</title>
    <style>
        :root {{
            --primary-color: #8B0000; /* Changed to dark red */
            --secondary-color: #B22222; /* Changed to firebrick red */
            --accent-color: #ffd700;
            --success-color: #28a745;
            --warning-color: #ffc107;
            --danger-color: #dc3545;
            --light-bg: #f8f9fa;
            --dark-text: #343a40;
        }}
        
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #8B0000 0%, #B22222 100%); /* Changed to red gradient */
            color: var(--dark-text);
            line-height: 1.6;
        }}
        
        .report-container {{
            max-width: 1400px;
            margin: 40px auto;
            background: white;
            border-radius: 20px;
            overflow: hidden;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
        }}
        
        .report-header {{
            background: linear-gradient(90deg, var(--primary-color), var(--secondary-color));
            color: white;
            padding: 40px;
            text-align: center;
            position: relative;
            overflow: hidden;
        }}
        
        .report-header::before {{
            content: '⚽';
            position: absolute;
            font-size: 300px;
            opacity: 0.1;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
        }}
        
        .report-title {{
            font-size: 3em;
            margin-bottom: 10px;
            position: relative;
            z-index: 1;
            color: #FFFFFF; /* White color for contrast */
            text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5); /* Added shadow for better readability */
            font-weight: bold;
        }}
        
        .report-subtitle {{
            font-size: 1.2em;
            opacity: 0.9;
            font-weight: 300;
            position: relative;
            z-index: 1;
            color: #FFD700; /* Gold color for subtitle */
        }}
        
        .champion-section {{
            padding: 40px;
            text-align: center;
            background: linear-gradient(90deg, #fff9c4, #fffde7);
            border-bottom: 5px solid var(--accent-color);
        }}
        
        .champion-title {{
            font-size: 2em;
            color: var(--primary-color);
            margin-bottom: 20px;
        }}
        
        .champion-card {{
            display: inline-block;
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.1);
            margin: 20px;
            min-width: 300px;
            border: 3px solid var(--accent-color);
        }}
        
        .champion-name {{
            font-size: 2.5em;
            color: var(--primary-color);
            margin: 10px 0;
        }}
        
        .champion-stats {{
            display: flex;
            justify-content: center;
            gap: 30px;
            margin-top: 20px;
            flex-wrap: wrap;
        }}
        
        .stat-card {{
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 5px 15px rgba(0,0,0,0.05);
            min-width: 200px;
            text-align: center;
            border-top: 4px solid var(--secondary-color);
        }}
        
        .stat-value {{
            font-size: 2em;
            font-weight: bold;
            color: var(--secondary-color);
            margin-bottom: 5px;
        }}
        
        .stat-label {{
            color: #6c757d;
            font-size: 0.9em;
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        
        .prediction-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(300px, 1fr));
            gap: 20px;
            padding: 40px;
        }}
        
        .prediction-card {{
            background: white;
            border-radius: 10px;
            padding: 25px;
            box-shadow: 0 5px 15px rgba(0,0,0,0.05);
            border-left: 4px solid var(--primary-color);
            transition: transform 0.3s ease;
        }}
        
        .prediction-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 10px 25px rgba(0,0,0,0.1);
        }}
        
        .prediction-title {{
            color: var(--primary-color);
            margin-bottom: 15px;
            padding-bottom: 10px;
            border-bottom: 2px solid var(--light-bg);
            display: flex;
            align-items: center;
            gap: 10px;
        }}
        
        .prediction-list {{
            list-style: none;
        }}
        
        .prediction-item {{
            padding: 10px 0;
            border-bottom: 1px solid var(--light-bg);
            display: flex;
            justify-content: space-between;
            align-items: center;
        }}
        
        .prediction-item:last-child {{
            border-bottom: none;
        }}
        
        .team-name {{
            font-weight: 500;
        }}
        
        .probability {{
            background: var(--light-bg);
            padding: 5px 15px;
            border-radius: 20px;
            font-weight: bold;
            color: var(--primary-color);
        }}
        
        .probability.high {{ background: #d4edda; color: #155724; }}
        .probability.medium {{ background: #fff3cd; color: #856404; }}
        .probability.low {{ background: #f8d7da; color: #721c24; }}
        
        .methodology {{
            background: var(--light-bg);
            padding: 40px;
            margin: 20px;
            border-radius: 15px;
        }}
        
        .methodology h3 {{
            color: var(--primary-color);
            margin-bottom: 20px;
        }}
        
        .footer {{
            background: var(--primary-color);
            color: white;
            padding: 30px;
            text-align: center;
            margin-top: 40px;
        }}
        
        @media (max-width: 768px) {{
            .report-title {{ font-size: 2em; }}
            .champion-name {{ font-size: 1.8em; }}
            .prediction-grid {{ grid-template-columns: 1fr; }}
        }}
    </style>
</head>
<body>
    <div class="report-container">
        <div class="report-header">
            <h1 class="report-title" style="color: #FFFFFF !important;">FIFA World Cup 2026 Prediction Report</h1>
            <p class="report-subtitle">Comprehensive analysis based on team ratings, simulations, and machine learning</p>
        </div>
        
        <div class="champion-section">
            <h2 class="champion-title">🏆 Predicted Champion</h2>
            <div class="champion-card">
                <div class="champion-name">{champion}</div>
                <div style="color: #6c757d; margin-bottom: 20px;">Based on {simulations} simulations</div>
                
                <div class="champion-stats">
                    <div class="stat-card">
                        <div class="stat-value">{champion_prob}%</div>
                        <div class="stat-label">Win Probability</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-value">{champion_rating}</div>
                        <div class="stat-label">Team Rating</div>
                    </div>
                    <div class="stat-card">
                        <div class="stat-value">{runner_up}</div>
                        <div class="stat-label">Runner-up</div>
                    </div>
                </div>
            </div>
        </div>
        
        <div class="prediction-grid">
            <div class="prediction-card">
                <h3 class="prediction-title">📊 Predicted Group Winners</h3>
                <ul class="prediction-list">
                    {group_winners}
                </ul>
            </div>
            
            <div class="prediction-card">
                <h3 class="prediction-title">🎯 Top 5 Championship Contenders</h3>
                <ul class="prediction-list">
                    {top_contenders}
                </ul>
            </div>
            
            <div class="prediction-card">
                <h3 class="prediction-title">🌟 Potential Surprises (Dark Horses)</h3>
                <ul class="prediction-list">
                    {dark_horses}
                </ul>
            </div>
            
            <div class="prediction-card">
                <h3 class="prediction-title">📈 Tournament Statistics</h3>
                <ul class="prediction-list">
                    <li class="prediction-item">
                        <span class="team-name">Total Teams</span>
                        <span class="probability">{total_teams}</span>
                    </li>
                    <li class="prediction-item">
                        <span class="team-name">Average Team Rating</span>
                        <span class="probability">{avg_rating}</span>
                    </li>
                    <li class="prediction-item">
                        <span class="team-name">Strongest Team</span>
                        <span class="probability">{strongest_team}</span>
                    </li>
                    <li class="prediction-item">
                        <span class="team-name">Most Balanced Group</span>
                        <span class="probability">{balanced_group}</span>
                    </li>
                </ul>
            </div>
        </div>
        
        <div class="methodology">
            <h3>🔬 Methodology</h3>
            <p>This prediction report is based on multiple analytical approaches:</p>
            <ul style="margin-left: 20px; margin-top: 15px;">
                <li><strong>Team Ratings:</strong> Based on Elo rating system with adjustments</li>
                <li><strong>Group Stage Simulation:</strong> Monte Carlo simulation of all group matches</li>
                <li><strong>Knockout Stage:</strong> Probabilistic model with pressure factors</li>
                <li><strong>Machine Learning:</strong> Random Forest classifier trained on historical patterns</li>
                <li><strong>Monte Carlo:</strong> 500 simulations for probability calculations</li>
            </ul>
            <p style="margin-top: 20px; font-style: italic;">
                Note: Predictions are based on current data and may change due to injuries, form, and other factors.
            </p>
        </div>
        
        <div class="footer">
            <p>Generated on {generation_date} | World Cup 2026 Prediction System</p>
            <p style="margin-top: 10px; opacity: 0.8;">
                Disclaimer: These predictions are for analytical purposes only.
            </p>
        </div>
    </div>
</body>
</html>
"""

# Fill the HTML template with all the calculated variables
final_report = final_report_html.format(
    champion=champion_name,
    simulations=simulations_count,
    champion_prob=f"{champion_prob_val:.1f}",
    champion_rating=champion_rating_val,
    runner_up=runner_up_name,
    group_winners=group_winners_html,
    top_contenders=top_contenders_html,
    dark_horses=dark_horses_html,
    total_teams=len(all_teams),
    avg_rating=avg_rating_value,
    strongest_team=strongest_team_display,
    balanced_group=f"Group {most_balanced_group}",
    generation_date=generation_date
)

# Save final report
with open('world_cup_2026_final_report.html', 'w', encoding='utf-8') as f:
    f.write(final_report)

print("✅ Final report generated successfully!")
print("📁 HTML reports saved:")
print("   1. 'qualified_teams_2026.html' - Group stage qualification")
print("   2. 'tournament_bracket_2026.html' - Tournament bracket")
print("   3. 'world_cup_2026_final_report.html' - Complete analysis report")

# Display summary
print("\n📊 SUMMARY OF PREDICTIONS:")
print(f"   • Most Likely Champion: {champion_name}")
print(f"   • Probability: {champion_prob_val:.1f}%")
if len(top_contenders_list) > 1:
    print(f"   • Top Contender: {top_contenders_list[1][0]}")
else:
    print(f"   • Top Contender: Unknown")
if dark_horses_list and len(dark_horses_list) > 0:
    print(f"   • Dark Horse: {dark_horses_list[0][0]}")
else:
    print(f"   • Dark Horse: None")
print(f"   • Strongest Team: {strongest_team_display}")
print(f"   • Total Teams Analyzed: {len(all_teams)}")

# Display in notebook
from IPython.display import display, HTML
display(HTML(final_report))

print("\n" + "="*70)
print("🎉 WORLD CUP 2026 PREDICTION SYSTEM COMPLETE!")
print("="*70)

FINAL REPORT - WORLD CUP 2026 PREDICTIONS
✅ Final report generated successfully!
📁 HTML reports saved:
   1. 'qualified_teams_2026.html' - Group stage qualification
   2. 'tournament_bracket_2026.html' - Tournament bracket
   3. 'world_cup_2026_final_report.html' - Complete analysis report

📊 SUMMARY OF PREDICTIONS:
   • Most Likely Champion: Brazil
   • Probability: 25.0%
   • Top Contender: France
   • Dark Horse: None
   • Strongest Team: Brazil (2100.0)
   • Total Teams Analyzed: 48



🎉 WORLD CUP 2026 PREDICTION SYSTEM COMPLETE!
